# Plan-and-Execute

This notebook shows how to create a "plan-and-execute" style agent. This is heavily inspired by the [Plan-and-Solve](https://arxiv.org/abs/2305.04091) paper as well as the [Baby-AGI](https://github.com/yoheinakajima/babyagi) project.

The core idea is to first come up with a multi-step plan, and then go through that plan one item at a time.
After accomplishing a particular task, you can then revisit the plan and modify as appropriate.

This compares to a typical [ReAct](https://arxiv.org/abs/2210.03629) style agent where you think one step at a time.
The advantages of this "plan-and-execute" style agent are:

1. Explicit long term planning (which even really strong LLMs can struggle with)
2. Ability to use smaller/weaker models for the execution step, only using larger/better models for the planning step

## Setup

First, we need to install the packages required.

```bash
npm install @langchain/langgraph @langchain/openai langchain
```

Next, we need to set API keys for OpenAI (the LLM we will use) and Tavily (the search tool we will use)

In [3]:
// Deno.env.set("OPENAI_API_KEY", "YOUR_API_KEY")
// Deno.env.set("TAVILY_API_KEY", "YOUR_API_KEY")

Optionally, we can set API key for LangSmith tracing, which will give us best-in-class observability.

In [4]:
// Deno.env.set("LANGCHAIN_TRACING_V2", "true")
// Deno.env.set("LANGCHAIN_API_KEY", "YOUR_API_KEY")
// Deno.env.set("LANGCHAIN_PROJECT", "YOUR_PROJECT_NAME")

## Define Tools

We will first define the tools we want to use. For this simple example, we will use a built-in search tool via Tavily. However, it is really easy to create your own tools - see documentation [here](https://js.langchain.com/docs/modules/agents/tools/dynamic) on how to do that.

In [5]:
import { TavilySearchResults } from "@langchain/community/tools/tavily_search";

const tools = [new TavilySearchResults({ maxResults: 3 })];

## Define our Execution Agent

Now we will create the execution agent we want to use to execute tasks. 
Note that for this example, we will be using the same execution agent for each task, but this doesn't HAVE to be the case.

In [6]:
import { pull } from "langchain/hub";
import { ChatPromptTemplate } from "@langchain/core/prompts";
import { ChatOpenAI } from "@langchain/openai";
import { createOpenAIFunctionsAgent } from "langchain/agents";
// Get the prompt to use - you can modify this!
const prompt = await pull<ChatPromptTemplate>("hwchase17/openai-functions-agent");
// Choose the LLM that will drive the agent
const llm = new ChatOpenAI({ modelName: "gpt-4-0125-preview" })
// Construct the OpenAI Functions agent
const agentRunnable = await createOpenAIFunctionsAgent({
  llm, tools, prompt
});

[WARNING]: Importing from "langchain/embeddings/fake" is deprecated.

Instead, please import from "@langchain/core/utils/testing".

This will be mandatory after the next "langchain" minor version bump to 0.2.


In [7]:
import { createAgentExecutor } from "@langchain/langgraph/prebuilt";

const agentExecutor = createAgentExecutor({
  agentRunnable,
  tools,
});

In [9]:
await agentExecutor.invoke({ input: "who is the winnner of the us open" });

Skipping write for channel chatHistory which has no readers


TypeError: Cannot use 'in' operator to search for 'messageLog' in undefined